In [95]:
import numpy as np
import pandas as pd
import regex
from collections import Counter
from tqdm import tqdm

In [ ]:
users = pd.read_csv('../data/users.csv')

In [8]:
users.isnull().sum(axis=0)

user_id         0
location        0
age         27833
dtype: int64

## context_data

In [ ]:
def str2list(x: str) -> list:
    '''문자열을 리스트로 변환하는 함수'''
    return x[1:-1].split(', ')

def split_location(x: str) -> list:
    '''
    Parameters
    ----------
    x : str
        location 데이터

    Returns
    -------
    res : list
        location 데이터를 나눈 뒤, 정제한 결과를 반환합니다.
        순서는 country, state, city, ... 입니다.
    '''
    res = x.split(',')
    res = [i.strip().lower() for i in res]
    res = [regex.sub(r'[^a-zA-Z/ ]', '', i) for i in res]  # remove special characters
    res = [i if i not in ['n/a', ''] else np.nan for i in res]  # change 'n/a' into NaN
    res.reverse()  # reverse the list to get country, state, city, ... order

    # for i in range(len(res)-1, 0, -1):
    #     if (res[i] in res[:i]) and (not pd.isna(res[i])):  # remove duplicated values if not NaN
    #         res.pop(i)

    return res

In [ ]:
users_ = users.copy()

In [238]:
users_['age'] = users_['age'].fillna(users_['age'].mode()[0]) # 29세로 결측치 대체 (27833 rows)
users_['age_range'] = users_['age'].apply(lambda x: x // 10 * 10)  # 10대, 20대, 30대, ...

users_['location_list'] = users_['location'].apply(lambda x: split_location(x)) 
users_['location_country'] = users_['location_list'].apply(lambda x: x[0])
users_['location_state'] = users_['location_list'].apply(lambda x: x[1] if len(x) > 1 else np.nan)
users_['location_city'] = users_['location_list'].apply(lambda x: x[2] if len(x) > 2 else np.nan)

### 주소 입력 오류 수정

1. city와 country가 같은데 state가 다른 케이스

In [240]:
# city와 country가 같은 케이스 찾기
city_country_pairs = users_.dropna(subset = 'location_state')
city_country_pairs = city_country_pairs.groupby(['location_city', 'location_country'])['location_state'].nunique()[users_.groupby(['location_city', 'location_country'])['location_state'].nunique() > 1].reset_index().dropna().sort_values('location_state')

In [ ]:
# city와 country가 같은 경우에 state가 여러개 존재함 -> 입력 오류로 판단
city_country_pairs

,location_city,location_country,location_state
33,alpine,usa,2
34,alta,usa,2
37,alverca,portugal,2
38,amboy,usa,2
1729,wuppertal,germany,2
...,...,...,...
1453,springfield,usa,13
325,clinton,usa,14
1626,washington,usa,15
99,barcelona,spain,19


In [ ]:
# 최빈값으로 대체하면 될 듯
users_[(users_.location_city == 'london') & (users_.location_country == 'united kingdom')][['location_country', 'location_state', 'location_city']].value_counts()

location_country  location_state  location_city
united kingdom    england         london           415
                  uk              london             6
                  middlesex       london             4
                  greater london  london             3
                  lazio           london             2
                  alabama         london             1
                  alaska          london             1
                  battersea       london             1
                  city            london             1
                  bayern          london             1
                  brixton         london             1
                  forest gate     london             1
                  foreign         london             1
                  chiswickl       london             1
                  muswell hill    london             1
                  none            london             1
                  scotland        london             1
                 

In [241]:
# 각 city-country 쌍에 대해 최빈 state 찾기
for _, row in tqdm(city_country_pairs.iterrows()):
    city = row['location_city']
    country = row['location_country']
    
    # 해당 city-country 조합의 모든 데이터 찾기
    condition = (users_['location_city'] == city) & (users_['location_country'] == country) 
    states = users_[condition]['location_state'].dropna()
    
    # 최빈 state 찾기
    if not states.empty:
        most_common_state = Counter(states).most_common(1)[0][0]
        
        # 최빈 state로 업데이트
        users_.loc[condition, 'location_state'] = most_common_state

1826it [00:20, 87.63it/s]


2. city와 state가 같은데 country가 다른 케이스

In [ ]:
# 2. city와 state가 같은 케이스 찾기
state_city_pairs = users_.dropna(subset = 'location_country')
state_city_pairs = state_city_pairs.groupby(['location_city', 'location_state'])['location_country'].nunique()[users_.groupby(['location_city', 'location_state'])['location_country'].nunique() > 1].reset_index().dropna().sort_values('location_country')

In [243]:
# 각 city-country 쌍에 대해 최빈 state 찾기
for _, row in tqdm(state_city_pairs.iterrows()):
    city = row['location_city']
    state = row['location_state']
    
    # 해당 city-country 조합의 모든 데이터 찾기
    condition = (users_['location_city'] == city) & (users_['location_state'] == state)
    country = users_[condition]['location_country'].dropna()
    
    # 최빈 state 찾기
    if not country.empty:
        most_common_country = Counter(country).most_common(1)[0][0]
        
        # 최빈 state로 업데이트
        users_.loc[condition, 'location_country'] = most_common_country

119it [00:01, 84.21it/s]


3. 나머지 결측치 최빈값 대체 

In [244]:
# 나머지 결측치는 최빈값으로 대체
for idx, row in users_.iterrows():
    if (not pd.isna(row['location_state'])) and pd.isna(row['location_country']): # state는 있고 country는 없는 경우 
        fill_country = users_[users_['location_state'] == row['location_state']]['location_country'].mode()
        fill_country = fill_country[0] if len(fill_country) > 0 else np.nan
        users_.loc[idx, 'location_country'] = fill_country
    elif (not pd.isna(row['location_city'])) and pd.isna(row['location_state']): # city는 있고 state는 없는 경우
        if not pd.isna(row['location_country']): # country는 있는 경우 
            fill_state = users_[(users_['location_country'] == row['location_country']) 
                                & (users_['location_city'] == row['location_city'])]['location_state'].mode()
            fill_state = fill_state[0] if len(fill_state) > 0 else np.nan
            users_.loc[idx, 'location_state'] = fill_state
        else: # country는 없는 경우
            fill_state = users_[users_['location_city'] == row['location_city']]['location_state'].mode()
            fill_state = fill_state[0] if len(fill_state) > 0 else np.nan
            fill_country = users_[users_['location_city'] == row['location_city']]['location_country'].mode()
            fill_country = fill_country[0] if len(fill_country) > 0 else np.nan
            users_.loc[idx, 'location_country'] = fill_country
            users_.loc[idx, 'location_state'] = fill_state

In [232]:
users_.isnull().sum(axis=0)

user_id                0
location               0
age                    0
age_range              0
location_list          0
location_country     143
location_state       725
location_city       2787
dtype: int64

In [ ]:
users_[users_.location_city.isna()] # 도시를 state에도 쓴 경우네 (베이스라인 중복 제거 코드 때문인 듯)

,user_id,location,age,age_range,location_list,location_country,location_state,location_city
2,11676,"n/a, n/a, n/a",29.0,20.0,"[nan, nan, nan]",NaN,NaN,NaN
88,13582,"lisboa, lisboa, portugal",29.0,20.0,"[portugal, lisboa]",portugal,lisboa,NaN
96,20801,"rio de janeiro, rio de janeiro, brazil",29.0,20.0,"[brazil, rio de janeiro]",brazil,rio de janeiro,NaN
131,48268,"vienna, vienna, austria",18.0,10.0,"[austria, vienna]",austria,vienna,NaN
141,57108,"new york, new york, usa",38.0,30.0,"[usa, new york]",usa,new york,NaN
...,...,...,...,...,...,...,...,...
68053,275892,"82061, calabria, germany",43.0,40.0,"[germany, calabria, nan]",germany,calabria,NaN
68057,276216,"ccp, ccp, chile",29.0,20.0,"[chile, ccp]",chile,ccp,NaN
68060,276442,"genève, genève, switzerland",62.0,60.0,"[switzerland, genve]",switzerland,genve,NaN
68063,276660,"singapore, n/a, singapore",15.0,10.0,"[singapore, nan]",singapore,NaN,NaN


In [245]:
# 중복 제거 코드 제외하면
users_.isnull().sum(axis=0)

user_id               0
location              0
age                   0
age_range             0
location_list         0
location_country    140
location_state      528
location_city       186
dtype: int64